# Decentralized Auction-Based Scheduling — POC

This notebook demonstrates a decentralized-style scheduling method where depots propose local schedules,
then use a simple auction mechanism to allocate shared vehicles. It illustrates:
1. Local scheduling heuristics
2. Lightweight negotiation (auction) for shared assets
3. Dynamic rescheduling on new order arrivals


In [1]:
# Imports & Scenario
import numpy as np
import random
from math import hypot
random.seed(0); np.random.seed(0)


In [2]:
# Scenario Setup
NUM_DEPOTS = 3
NUM_SHARED_VEHICLES = 4
NUM_STATIC_VEHICLES_PER_DEPOT = 1
NUM_ORDERS_INITIAL = 9
AREA_SIZE = 100
depots = [(20, 20), (80, 20), (50, 80)]
shared_vehicles = [{'pos': (50,50), 'id': f'S{idx}', 'assigned': None} for idx in range(NUM_SHARED_VEHICLES)]
local_vehicles = []
for d in range(NUM_DEPOTS):
    for v in range(NUM_STATIC_VEHICLES_PER_DEPOT):
        local_vehicles.append({'depot': d, 'pos': depots[d], 'id': f'L{d}_{v}'})
def rand_point_around(center, spread=10):
    cx, cy = center
    return (cx + random.uniform(-spread, spread), cy + random.uniform(-spread, spread))
orders = []
for i in range(NUM_ORDERS_INITIAL):
    depot_id = i % NUM_DEPOTS
    pickup = rand_point_around(depots[depot_id], spread=12)
    drop = rand_point_around((random.uniform(0,AREA_SIZE), random.uniform(0,AREA_SIZE)), spread=15)
    orders.append({'id': f'O{i}', 'depot': depot_id, 'pickup': pickup, 'drop': drop, 'created': 0, 'assigned_vehicle': None})
print("Depots:", depots)
print("Initial orders:", [o['id'] for o in orders])


Depots: [(20, 20), (80, 20), (50, 80)]
Initial orders: ['O0', 'O1', 'O2', 'O3', 'O4', 'O5', 'O6', 'O7', 'O8']


In [3]:
# Helpers & Scheduling
def dist(a,b): return hypot(a[0]-b[0], a[1]-b[1])
def route_cost_for_order(vehicle_pos, pickup, drop):
    return dist(vehicle_pos, pickup) + dist(pickup, drop)
def local_schedule(depot_id, orders_pool, local_vehicles, shared_allocations):
    proposals = []
    loc_vs = [v for v in local_vehicles if v['depot']==depot_id]
    unassigned = [o for o in orders_pool if o['depot']==depot_id and o['assigned_vehicle'] is None]
    for v in loc_vs:
        if not unassigned: break
        o = unassigned.pop(0)
        cost = route_cost_for_order(v['pos'], o['pickup'], o['drop'])
        proposals.append({'order': o, 'vehicle': v['id'], 'marginal_cost': cost, 'vehicle_obj': v, 'type':'local'})
    for o in unassigned:
        best_shared=None; best_cost=1e9; best_obj=None
        for sv in shared_vehicles:
            if sv['assigned'] and sv['assigned']!=depot_id:
                continue
            c = route_cost_for_order(sv['pos'], o['pickup'], o['drop'])
            if c < best_cost:
                best_cost = c; best_shared = sv['id']; best_obj=sv
        if best_shared:
            proposals.append({'order': o, 'vehicle': best_shared, 'marginal_cost': best_cost, 'vehicle_obj': best_obj, 'type':'shared'})
    return proposals
def run_auction(all_proposals):
    winners={}
    for p in all_proposals:
        oid=p['order']['id']
        if oid not in winners or p['marginal_cost']<winners[oid]['marginal_cost']:
            winners[oid]=p
    for oid,p in winners.items():
        o=p['order']; vobj=p['vehicle_obj']
        o['assigned_vehicle']=vobj['id']
        if p['type']=='shared': vobj['assigned']=o['depot']
    return list(winners.values())
def inject_new_orders(tim, count=2):
    start_idx = len(orders)
    for i in range(count):
        depot_id = random.randrange(NUM_DEPOTS)
        pickup = rand_point_around(depots[depot_id], spread=12)
        drop = rand_point_around((random.uniform(0,AREA_SIZE), random.uniform(0,AREA_SIZE)), spread=15)
        orders.append({'id': f'O{start_idx+i}', 'depot': depot_id, 'pickup': pickup, 'drop': drop, 'created': tim, 'assigned_vehicle': None})


In [4]:
# Simulation
def simulate(time_horizon=10, inject_times=[3,6]):
    metrics={'total_cost':0,'assignments':0}
    for t in range(time_horizon):
        if t in inject_times:
            inject_new_orders(t, count=2)
        all_props=[]
        for d in range(NUM_DEPOTS):
            props=local_schedule(d, orders, local_vehicles, shared_vehicles)
            all_props.extend(props)
        winners=run_auction(all_props)
        metrics['assignments'] += len(winners)
        for w in winners:
            metrics['total_cost'] += w['marginal_cost']
        assigned_count = sum(1 for o in orders if o['assigned_vehicle'])
        print(f"t={t}: orders={len(orders)} assigned={assigned_count} total_cost={metrics['total_cost']:.1f}")
    return metrics

metrics = simulate()
print("Done", metrics)


t=0: orders=9 assigned=9 total_cost=774.5
t=1: orders=9 assigned=9 total_cost=774.5
t=2: orders=9 assigned=9 total_cost=774.5
t=3: orders=11 assigned=11 total_cost=944.1
t=4: orders=11 assigned=11 total_cost=944.1
t=5: orders=11 assigned=11 total_cost=944.1
t=6: orders=13 assigned=13 total_cost=1115.7
t=7: orders=13 assigned=13 total_cost=1115.7
t=8: orders=13 assigned=13 total_cost=1115.7
t=9: orders=13 assigned=13 total_cost=1115.7
Done {'total_cost': 1115.7299533822916, 'assignments': 13}


### Notes & Extensions
- This POC uses a central auctioneer for simplicity; in practice, the auction messages could be passed peer-to-peer (decentralized).
- Extensions: richer bids (include time windows, penalties), repeated auctions, and privacy-aware message passing.
- This demonstrates decentralised scheduling logic and dynamic rescheduling for shared assets.
